[![](https://bytebucket.org/davis68/resources/raw/f7c98d2b95e961fae257707e22a58fa1a2c36bec/logos/baseline_cse_wdmk.png?token=be4cc41d4b2afe594f5b1570a3c5aad96a65f0d6)](http://cse.illinois.edu/)

# Threaded Programming for Engineers (OpenMP)

<a id='contents'></a>
## Contents
- [Threaded Programs](#intro)
- [Hello World!](#hello)
- [Numerical Integration](#numint)
- [Recursion (Fibonacci Sequence)](#fib)
- [Vector & Matrix Multiplication](#mmult)
- [Series Formulæ](#series)
- [Monte Carlo](#montecarlo)
- [A Few More Tips](#tips)
- [Using Vectorization (SIMD) in Practice](#vector)
- [Assessing Performance](#assess)
- [Best Practices for Scientific Parallel Programming](#bestprac)
- [Conclusion](#conc)
- [References](#refs)
- [Credits](#credits)

<a id='intro'></a>
## Threaded Programs

[OpenMP](http://www.openmp.org/) is an interface for C/C++/Fortran to direct multithreaded, shared-memory parallel execution.  OpenMP allows a group of processors to coördinate work between themselves using the _fork–join_ model of parallelism.  In this model, a _master_ thread (or process) spawns groups of _slave_ threads as appropriate to solve certain calculations.

![](https://bytebucket.org/davis68/parallel/raw/fc24d274c088ba5087688ef25cf8e724f532319d/lessons/img/fork-join.png?token=8f9936a462a4862f1a2894d28b33ce7aa57e7cb4)

An example of a parallel addition in this model is shown below.  Note that, in OpenMP terms, separate processes could be spawned to perform each addition—this differs from _vectorization_ in which a single process would perform the operation as a single step across multiple vector elements ([ref](https://stackoverflow.com/questions/10509872/comparison-between-openmp-and-vectorization)).  (More on that distinction—and how to exploit both to your advantage—below.)

![](https://bytebucket.org/davis68/parallel/raw/fc24d274c088ba5087688ef25cf8e724f532319d/lessons/img/SIMD-vector-add.png?token=7d71987d20cfb22695d0444a321a9d9480610c6f)

The interface for working with OpenMP is reasonably straightforward (in comparison to MPI), as it is implemented entirely with `#pragma omp`s in C and C++ and `!$OMP` sentinels in Fortran.  Thus existing serial code can be naïvely parallelized with almost no effort (although this guarantees no improvement in performance without tweaks).

---
<a id='hello'></a>
## Hello World

Let's take a look at a trivial C++ code which uses OpenMP.

As we are using an [IPython](ipython.org) notebook interface to unite code and context, simply select each cell and press Shift+Enter to run the following three blocks of code in succession.
1. The first writes the C++ code in it to disk as the `openmp.cpp`.
1. The second compiles the C++ code.
1. The third executes the code in a `bash` shell and echoes the output back in the notebook.

In [ ]:
%%file openmp.cpp

#include <iostream>
#include <omp.h>
using namespace std;
 
int main(int argc, char *argv[]) {
    int th_id, nthreads;

    #pragma omp parallel private(th_id) shared(nthreads)
    {
        th_id = omp_get_thread_num();
        
        cout << "Hello World from thread " << th_id << '\n';
        #pragma omp barrier
        
        #pragma omp single
        cout << "---" << endl;
        
        #pragma omp critical
        cout << "Hello World from thread " << th_id << '\n';
        
        #pragma omp barrier

        cout << "There are " << omp_get_max_threads() << " threads" << '\n';
    }

    return 0;
}

In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  openmp.cpp -o openmp

In [ ]:
!./openmp

There's actually several things going on there, so let's unpack the output first and then the code body to get a taste of OpenMP.

In [ ]:
# Timing code in IPython
%timeit !./openmp

### OpenMP `#pragma`s

What would happen if we forgot the `-fopenmp` flag?  Well, `#pragma`s are preprocessor commands to the compiler to invoke certain features, and thus those features (in this case OpenMP) wouldn't be invoked.  In some cases, like the "Hello World" example, this _may_ make no difference, but that's hardly the general case.  Hey, let's just try it:

In [ ]:
!g++ -Wall -O2 -pedantic  openmp.cpp -o openmp_nopragma

---
<a id='numint'></a>
## Numerical Integration
Let's do something more interesting, assuming that you find writing parallel numerical methods interesting :).  Consider the following decomposition of the composite Simpson's rule for integral evaluation.

$$I = \int_{a}^{b} \textrm{d}x\, f(x)$$

$$
T[f] = \Delta x \left(
\frac{1}{2} f_{\textrm{0}} + \frac{1}{2} f_{n} + \sum_{j=1}^{n-1} f_{j}
\right)
\approx
I
$$

Consider the integral
$$
\int_{\pi}^{5\pi/2}
\textrm{d}x\,
\frac{\cos(x)}{2 + \cos(x)}
$$
which is depicted below.

![](https://bytebucket.org/davis68/parallel/raw/fc24d274c088ba5087688ef25cf8e724f532319d/lessons/img/int-cosby2.png?token=91e032a5d2586d8a80d37c03adbe5a5e5072f20a)

First, let's look at a serial implementation of the trapezoid rule in C.

In [ ]:
%%file int_trap_serial.cpp
#include <iostream>
#include <cmath>
using namespace std;

#define PI 3.14159265
#define N 1000

double integrand(double x) {
    return cos(x)/(2+cos(x));
}

int main(int argc, char** argv) {
    double result = 0,
           x;
    double a = PI,
           b = 5*PI/2;
    double dx = (b - a) / N;
    
    for (int j = 1; j < N; j++) {
        x = dx * j;
        result += integrand(x);
    }
    // Add endpoint calculations.
    result += 1/2 * integrand(a);
    result += 1/2 * integrand(b);
    // Multiply by prefactor
    result *= dx;
    
    cout << "The calculated value is " << result << ".\n";
    return 0;
}

In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  int_trap_serial.cpp -o int_trap_serial

In [ ]:
!./int_trap_serial

Now, let's convert the same code into OpenMP parallel code.

In [ ]:
%%file int_trap_omp.cpp

#include <iostream>
#include <cmath>
using namespace std;
#include <omp.h>

#define PI 3.14159265
#define N 1000

double integrand(double x) {
    return cos(x)/(2+cos(x));
}

int main(int argc, char** argv) {
    double result = 0,
           x;
    double a = PI,
           b = 5*PI/2;
    double dx = (b - a) / N;
    
    int j;
    #pragma omp parallel for private(x) reduction(+:result) schedule(static,1)
    for (j = 1; j < N; j++) {
        x = dx * j;
        result += integrand(x);
    }
    cout << "Using " << omp_get_max_threads() << " threads, ";
    
    // Add endpoint calculations.
    result += 1/2 * integrand(a);
    result += 1/2 * integrand(b);
    // Multiply by prefactor
    result *= dx;
    
    cout << "the calculated value is " << result << ".\n";
    return 0;
} 


In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic int_trap_omp.cpp -o int_trap_omp

In [ ]:
!./int_trap_omp

In this case, with every iterative calculation being essentially independent of the others, we have to add very little.  Specifically, we added this compound statement.  Let's deconstruct it a bit to see what OpenMP is doing.
    #pragma omp parallel for private(x) reduction(+:result) schedule(static,1)
    
- [`#pragma`](https://gcc.gnu.org/onlinedocs/cpp/Pragmas.html) instructs the compiler that a special category of compiler-specific instructions follows.  Thus `#pragma omp` invokes compilation with OpenMP support for the following code section.
- `parallel` begins a block of—you guessed it—parallel code.  It forms a team of parallel threads which begin executing the subsequent code block in parallel.
- `for` signals that the following block is a `for` loop.  The number of iterations must be known at run time (_i.e._, you can't alter `N` or skip some iterations.  They will be partitioned among the threads as defined by `schedule`.
- `private` indicates a list of variables (previously declared) which are considered to be private to each thread—that is, every thread has its own copy of the variable.
- `reduction` is an operation which takes a group of values and combines them by some well-defined operation like addition or multiplication.  `result` is the `double` which will contain the answer.
- `schedule` refers to how iterations are to be divided between threads.  `static,1` means that the iterations are divided into groups of size `1` and assigned to threads in round-robin fashion.

### Exercise
- Write a Simpson's rule integrator which solves the integral
$$
\int_{0}^{2}
\textrm{d}x\,
\frac{\sinh{x}}{\cosh{x}} - 1
\textrm{.}
$$

Recall that Simpson's rule is
$$
S[f] = \Delta x \left(
\frac{1}{2} f_{\textrm{0}} + \frac{1}{2} f_{n} + \sum_{j=1}^{n-1} f_{j}
\right)
\approx
I
\text{.}
$$

![](https://bytebucket.org/davis68/parallel/raw/fc24d274c088ba5087688ef25cf8e724f532319d/lessons/img/int-sinhbycosh.png?token=95b968dc849b4128087c5cc1dadeb1f45e035c14)

A skeleton code for this is provided below.

In [ ]:
%%file int_simp_omp.cpp

#include <iostream>
#include <cmath>
using namespace std;
#include <omp.h>

#define PI 3.14159265
#define N 1000

double integrand(double x) {
    double value;
    // TODO
    return value;
}

int main(int argc, char** argv) {
    double result = 0,
           x;
    double a = 0,
           b = 2;
    double dx = (b - a) / N;
    
    int j;
    #pragma omp parallel for private(x) reduction(+:result) schedule(static,1)
    for (j = 1; j < N; j++) {
        //TODO
    }
    cout << "Using " << omp_get_max_threads() << " threads, ";
    
    // Add endpoint calculations.
    // TODO
    // Multiply by prefactor
    // TODO
    
    cout << "the calculated value is " << result << ".\n";
    return 0;
} 


In [ ]:
!/usr/local/bin/g++-4.8 -Wall -fopenmp -O2 -std=c++11 -pedantic  int_simp_omp.cpp -o int_simp_omp

In [ ]:
!./int_simp_omp

---
<a id='fib'></a>
## Recursion (Fibonacci Sequence)

Next, let's examine a recursive code written in OpenMP to see a different kind of internal structure.  (This is a purely didactic code:  there are much better algorithms to calculate the Fibonacci sequence than a recursive algorithm, much less a parallel one.)

In [37]:
%%file fib_serial.cpp

#include <iostream>
#include <cmath>
using namespace std;

long fib(int n) {
    if (n < 2) return n;
    else {
        long x, y;
        x = fib(n-1);
        y = fib(n-2);
        return x + y;
    }
}

int main(int argc, char** argv) {
    int    n = atoi(argv[1]); // Note no error checking here.
    double F = fib(n);
    cout << "The " << n << "th Fibonacci number is " << F << ".\n";
    return 0;
}

Overwriting fib_serial.cpp


In [38]:
!/usr/local/bin/g++-4.8 -Wall -fopenmp -O2 -std=c++11 -pedantic fib_serial.cpp -o fib_serial

In [39]:
!./fib_serial 18
# Don't go higher than 40.

The 18th Fibonacci number is 2584.


In [42]:
%%file fib_omp.cpp

#include <iostream>
#include <cmath>
using namespace std;
#include <omp.h>
#include <cstdlib>

long fib(int n) {
    if (n < 2)
        return n;
    else {
        long x, y;
        #pragma omp task shared(x)
        x=fib(n-1);
        #pragma omp task shared(y)
        y=fib(n-2);
        #pragma omp taskwait
        return x+y;
    }
}

int main(int argc, char** argv) {
    int    n = atoi(argv[1]); // Note no error checking here.
    double F;
    #pragma omp parallel
    F = fib(n);
    cout << "The " << n << "th Fibonacci number is " << F << ".\n";
    return 0;
}

Overwriting fib_omp.cpp


In [43]:
!g++ -Wall -fopenmp -O2 -pedantic  fib_omp.cpp -o fib_omp

In [45]:
!./fib_omp 18
# Don't go higher than 40.

The 18th Fibonacci number is 2584.


(As an aside, this code works fine without OpenMP.)

In [ ]:
!g++ -Wall -O2 -pedantic  fib_omp.cpp -o fib_nomp
!./fib_nomp 8

In [ ]:
%timeit !./fib_serial 18
%timeit !./fib_omp 18

In this case, we run the `fib` method inside of a `omp parallel` block.

This code forks new processes as children of the main (master) process to progressively solve the problem.  The total number of threads is still `OMP_MAX_THREADS`, though, so new processes spawned will get remapped back to the original processors.

As mentioned, this is not a good algorithm, nor is it more efficient in parallel.  It's merely a demonstration of a principle.

---
<a id='mmult'></a>
## Vector & Matrix Multiplication

An extremely common set of operations in scientific computing codes is vector–vector, vector–matrix, and matrix–matrix addition and multiplication.  (This says more about the way we do mathematical physics and what is straightforward to do on a computer than it does about the way the world works.)

Let's examine a simple vector–vector addition first, then look at some other cases.  In this case, we will actually use the SIMD model shown at first, which simply lets each thread perform its own addition.

![](https://bytebucket.org/davis68/parallel/raw/fc24d274c088ba5087688ef25cf8e724f532319d/lessons/img/SIMD-vector-add.png?token=7d71987d20cfb22695d0444a321a9d9480610c6f)

In [ ]:
%%file vadd.cpp
#include <iostream>
#include <cmath>
#include <vector>
using namespace std;
#include <omp.h>

#define N 16

int main(int argc, char** argv) {
    vector<double> a(N);
    vector<double> b(N);
    vector<double> c(N);
    
    // Initialize in parallel.
    # pragma omp parallel for
    for (int i = 0; i < a.size(); i++) {
        a[i] = i;
        b[i] = 2*i;
    }
    
    // Add in parallel.
    # pragma omp parallel for
    for (int i = 0; i < a.size(); i++) {
        c[i] = a[i] + b[i];
    }
    
    // Output result of calculation.
    for (int i = 0; i < a.size(); i++) {
        cout << a[i] << " + " << b[i] << "\t= " << c[i] << endl;
    }
}

In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  vadd.cpp -o vadd

In [ ]:
!./vadd

### Exercise
- Modify the program `vadd.cpp` to so each thread has its own copy of a single variable `i` declared _before_ the `#pragma omp` (hint:  use `private`) and use `dynamic` scheduling.  (You can copy it into the code block below to preserve a working example above.)

In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  vadd.cpp -o vadd

In [ ]:
!./vadd

### Exercise
- Write a vector–vector multiplication code in C (no STL but just C-style `double` arrays) using OpenMP.

In [ ]:
%%file vmult.c
#include <stdio.h>
#include <math.h>
#include <omp.h>

#define N 16

int main(int argc, char** argv) {
    double a[N][N];
    double b[N][N];
    double c[N][N];
    
    // your code here
}

In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  vmult.c -o vmult

In [ ]:
!./vmult

### Counterexample
Ultimately, the reason we can get away with working inside of an STL container is that we don't need threads to write to the same memory location, nor do we reallocate memory.  ([ref](https://stackoverflow.com/questions/9269097/openmp-and-stl-vector))  The following code, in contrast, will fail because the behavior of `push_back` requires a consistent memory location, violated by several threads competing for the same allocation.

In [ ]:
%%file vinit.cpp

#include <vector>
using namespace std;
#include <omp.h>

int main(int argc, char** argv) {
    vector<double> a(0);
    
    // Initialize a vector in parallel by adding new elements to it.
    # pragma omp parallel for
    for (int i = 0; i < 12; i++) {
        a.push_back(i);
    }
}

In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  vinit.cpp -o vinit

In [ ]:
!./vinit

### Matrix–Matrix Operations
Matrix–matrix operations are more involved since they often involve _blocking_ or _chunking_, which separates a large dense matrix into several smaller square pieces and solves them separately (with dependencies, of course).  The following C code illustrates this (original by Blaise Barney, LLNL, [src](https://computing.llnl.gov/tutorials/openMP/samples/C/omp_mm.c)).

In [ ]:
%%file mmult-block.c
//  Modified from an original example by Blaise Barney for LLNL.
//  https://computing.llnl.gov/tutorials/openMP/samples/C/omp_mm.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define NRA 62                 /* number of rows in matrix A */
#define NCA 15                 /* number of columns in matrix A */
#define NCB 7                  /* number of columns in matrix B */

int main (int argc, char *argv[]) {
    int tid, nthreads, i, j, k, chunk;
    double a[NRA][NCA],    /* matrix A to be multiplied */
           b[NCA][NCB],    /* matrix B to be multiplied */
           c[NRA][NCB];    /* result matrix C */
    
    chunk = 10;                    /* set loop iteration chunk size */
    
    #pragma omp parallel shared(a, b, c, nthreads, chunk) private(tid, i, j, k)
    {
        tid = omp_get_thread_num();
        if (tid == 0) {
            nthreads = omp_get_num_threads();
            printf("Starting matrix multiple example with %d threads\n", nthreads);
        }
        /*** Initialize matrices ***/
        #pragma omp for schedule (static, chunk) 
        for (i = 0; i < NRA; i++)
            for (j = 0; j < NCA; j++)
                a[i][j] = i + j;
        #pragma omp for schedule (static, chunk)
        for (i = 0; i < NCA; i++)
            for (j = 0; j < NCB; j++)
                b[i][j] = i * j;
        #pragma omp for schedule (static, chunk)
        for (i = 0; i < NRA; i++)
            for (j = 0; j < NCB; j++)
                c[i][j] = 0;
        
        /*** Do matrix multiply sharing iterations on outer loop ***/
        /*** Display who does which iterations for demonstration purposes ***/
        printf("Thread %d starting matrix multiply...\n",tid);
        #pragma omp for schedule (static, chunk)
            for (i = 0; i < NRA; i++) {
                printf("Thread=%d did row=%d\n",tid,i);
                for (j = 0; j < NCB; j++)       
                    for (k = 0; k < NCA; k++)
                        c[i][j] += a[i][k] * b[k][j];
            }
    } /*** end parallel section ***/
    
    printf("\nFirst Matrix:\n");
    for (i=0; i<NRA; i++) {
        for (j=0; j<NCA; j++)
            printf("%6.2f  ", a[i][j]);
        printf("\n"); 
    }
    printf("\nSecond Matrix:\n");
    for (i=0; i<NCA; i++) {
        for (j=0; j<NCB; j++)
            printf("%6.2f  ", b[i][j]);
        printf("\n"); 
    }
    
    printf("\nResult Matrix:\n");
    for (i=0; i<NRA; i++) {
        for (j=0; j<NCB; j++)
            printf("%6.2f  ", c[i][j]);
        printf("\n"); 
    }
    return 0;
}

In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  mmult-block.c -o mmult-block

In [ ]:
!./mmult-block

---
<a id='series'></a>
## Series Formulæ

Series representations are common useful forms for function approximations like trigonometric functions and spherical harmonics.  For instance, the sine function (for $z \in \mathbb{C}$) may be written in a product form as
$$\frac{\sin z}{z} = \prod_{k=1}^{\infty} \left( 1 - \frac{z^2}{k^2\pi^2} \right) \text{.}$$
For small $z$, this form converges quickly.  This being the case, we can calculate each term independently over $P$ processes and reduce the result by multiplication.

In [ ]:
%%file series.cpp
#include <complex>
#include <iostream>
#include <cmath>
#include <omp.h>

using namespace std;

#define PI 3.1415926535

// The k-th term of the infinite product series for sine.
static complex<double> term(int k, complex<double> z) {
    return (complex<double>(1.0, 0.0) - pow(z, 2)/(pow(k, 2) * PI*PI));
}

int main(int argc, char** argv) {
    int N = 10;
    if (argc > 1) N = atoi(argv[1]);
    complex<double> z = complex<double>(PI*0.25, PI*0.25);
    if (argc > 2) z = complex<double>(atof(argv[2]), atof(argv[3]));
    
    complex<double> res, i_res;
    double res_r = 1.0, res_i = 1.0;
    int j;
    #pragma omp parallel for private(i_res, j) reduction(*:res_r,res_i) schedule(static,1)
    for (j = 1; j < N; j++) {
        i_res = term(j, z);
        res_r *= real(i_res);
        res_i *= imag(i_res);
    }
    // note that reduction over STL data types is not yet supported
    res = complex<double>(res_r, res_i) * z;
    cout << "Using " << omp_get_max_threads() << " threads and " << N << " terms, ";
    cout << "the value of sin(" << z << ") is " << res << ".\n";
    cout << "This compares to the library value of " << sin(z) << " with an error of " << (sin(z)-res) << ".\n";
    return 0;
}

In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  series.cpp -o series

In [ ]:
!./series 16 0.0 0.31415

### Exercise
- Write a series summation code to calculate sine:
$$
\begin{eqnarray}
\sin x
& = x - \frac{x^3}{3!} + \frac{x^5}{5!} - \frac{x^7}{7!} + \cdots \\[8pt]
& = \sum_{n=0}^\infty \frac{(-1)^n}{(2n+1)!}x^{2n+1} \\
\end{eqnarray}
$$

Test it with $\sin (\pi/3) \approx 0.866025403784438...$.

In [ ]:
%%file sine.cpp


In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  sine.cpp -o sine

In [ ]:
!./sine 1.0471975511966  #pi/3

---
<a id='montecarlo'></a>
## Monte Carlo

Now, you'll probably never write your own matrix multiplication code (you should use a library instead), but you may very well find yourself needing to write a Monte Carlo code from scratch for a specific problem.  Monte Carlo problems use random numbers to sample an integral (or, effectively the same thing, integrate a differential equation) and can be very competitive for some classes of problems because they are often embarassingly parallel.

### Random Number Generation

The tricky thing about Monte Carlo codes is that the random numbers really do need to be independent.  As you are trying to explore some region of phase space thoroughly, even high-order repetitions can bias your results in subtle and difficult-to-discover ways.

[![](https://bytebucket.org/davis68/parallel/raw/fc24d274c088ba5087688ef25cf8e724f532319d/lessons/img/xkcd-random_number.png?token=35967e7728e79fb403034d834ccf92f127f4d215)](http://xkcd.com/)

We have previously glossed over this problem by simply invoking a uniform random number generator (RNG), $U \rightarrow x \in [0,1)$, that I hacked together because it works and is concise and readable without being distracting.

    double uniform_rng() { return (double)rand() / (double)RAND_MAX; } // Not good

Generally speaking, RNGs that are not reentrant (_i.e._, able to be accessed multiple times simultaneously and independently as well as interrupted and resumed indiscriminately) are not thread safe, and cannot be used in parallel code.  For a role-playing game, say, this is probably fine.  When exploring multidimensional phase space for a mission-critical activity, it's better to not take chances.  The _problem_ with this RNG, as I mentioned before, is that it is not _thread safe_.  That is, we can make no guarantees about whether or not subsequent numbers drawn from this distribution are in fact independent of prior numbers.  (Incidentally, I found this problem to be more pronounced with MPI than with OpenMP.)

In [ ]:
%%file rng_bad.cpp
#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <omp.h>

#define N 100000
#define R 1

double uniform_rng() { return (double)rand() / (double)RAND_MAX; }

int main(int argc, char *argv[]) {
    int tid;
    srand(time(NULL));
    
    //  Calculate and display a few "random" numbers.
    #pragma omp parallel private(tid)
    {
        tid = omp_get_thread_num();
        printf("%d\t%f\n", tid, uniform_rng());
    }
    
    return 0;
}

In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  rng_bad.cpp -o rng_bad
!./rng_bad

The next option you have is to use the RAND48 library which is a collection of POSIX standards and some associated GNU extensions for random number generation.  Below we illustrate `erand48`, which is [nominally thread safe](http://www.unix.com/man-page/POSIX/3posix/erand48/) although opinions vary.

In [ ]:
%%file rng_good.cpp
#include <cmath>
#include <cstdio>
#include <cstdlib>
#include <ctime>
#include <omp.h>

#define N 100000
#define R 1

int main(int argc, char *argv[]) {
    int tid;
    unsigned short rand_seed[3];
    srand48(time(NULL));
    
    //  Calculate and display a few random numbers.
    #pragma omp parallel private(tid, rand_seed)
    {
        tid = omp_get_thread_num();
        rand_seed[0] = tid + 120 + time(NULL);
        rand_seed[1] = tid * 2 + time(NULL);
        rand_seed[2] = tid * 100 + time(NULL);
        printf("%d\t%f\n", tid, erand48(rand_seed));
    }
    
    return 0;
}

In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  rng_good.cpp -o rng_good
!./rng_good

If we are using C++ instead of C, we can also `#include <random>` to access some thread-safe generators [(ref)](http://msdn.microsoft.com/en-us/library/bb982398.aspx).  (This is the same as the [Boost](http://www.boost.org/) `random` library, and we will make particular use of the Mersenne twister algorithm.)  We examine it first as a serial code to clarify what is going on, then as a parallel OpenMP code.

In [ ]:
%%file rng_better.cpp
#include <cmath>
#include <random>
#include <cstdio>
#include <cstdlib>
#include <ctime>
#include <omp.h>

#define N 100000
#define R 1

int main(int argc, char *argv[]) {
    int tid;
    std::random_device rd;   // non-deterministic bit generator
    std::mt19937 gen;        // Mersenne twister URNG
    std::uniform_real_distribution<> dist(0,1); // transforms results to a useful distribution and range
    
    //  Calculate and display a few random numbers.
    #pragma omp parallel private(tid, rd, gen, dist)
    {
        tid = omp_get_thread_num();
        gen.seed(time(NULL)+tid);
        
        printf("%d\t%f\n", tid, dist(gen));
    }
    
    return 0;
}

In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  rng_better.cpp -o rng_better
!./rng_better

Finally, the _best_ general option is to use a RNG library designed for parallel Monte Carlo simulations like [Tina's RNG](http://numbercrunch.de/trng/).  Since installation is specific to a system, we will forgo a demonstration here.  (Keep in mind that you can install libraries to your home folder though, even if the system administrators don't jump for a site installation.)

In [ ]:
# plotting code if you'd like to see RNGs v. rank.
from numpy import linspace, zeros, append
stdout = !./rng_better
x = []
y = []
for out in stdout:
    x.append(float(out.split('\t')[0]))
    y.append(float(out.split('\t')[1]))

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,4))
axes[0].plot(x, y, 'r.')
axes[1].plot(zeros(len(x)),y,'r.')
fig.show()

### Monte Carlo problem

That aside was longer than I had intended, but I hope it was profitable in laying out how to deal with random number generation safely and practically.  Let's take a look at our Monte Carlo problem again.

Consider the problem of determining, for the distance between a pair of random points, the average distance and standard deviation of that value.  The following code illustrates this principle.

In [ ]:
%%file monte_carlo_omp.cpp
#include <cmath>
#include <random>
#include <sys/time.h>
#include <cstdio>
#include <ctime>
#include <unistd.h>
#include <omp.h>

#define NPTS 10

using namespace std;

//  Termination criteria
double rtol       = 1e-2; // Relative tolerance
long   max_trials = 1e6;  // Maximum number of MC trials
long   num_batch  = 500;  // Number of batches per trial

/** is_converged()
 *  Use adaptive error estimation to terminate the simulation when the error bars
 *  at one standard deviation are below rtol.
 */
bool is_converged(double sum_x,          // Sum of x
                  double sum_x2,         // Sum of x, squared
                  double num_trials) {   // Number of trials
    double E_x   = sum_x / num_trials;
    double E_x2  = sum_x2 / num_trials;
    double var_x = E_x2 - E_x * E_x;
    return (var_x/(E_x*E_x) < rtol*rtol || num_trials > max_trials);
}

int main(int argc, char** argv) {
    int    num_threads = omp_get_max_threads();
    double E_x, E_x2, sigma_x;
    
    //  Monte Carlo result variables
    double all_sum_x   = 0;
    double all_sum_x2  = 0;
    long   all_ntrials = 0;
    
    //  Private state variables
    int    f_done;  // Flag for completion
    int    tid,     // ID of this OpenMP thread
           tnbatch; // 
    double sum_x,   // 
           sum_x2;  // 
    long   seed;    // Unique seed for Mersenne twister PRNG
    
    std::random_device rd;   // non-deterministic bit generator
    std::mt19937 gen;        // Mersenne twister URNG
    std::uniform_real_distribution<> dist(0,1); // transforms results to a useful distribution and range
    
    #pragma omp parallel shared(all_sum_x, all_sum_x2, all_ntrials, num_batch) private(gen, dist, seed, f_done, tid, sum_x, sum_x2)
    {
        tnbatch = num_batch;
        tid = omp_get_thread_num();
        f_done = false;
        
        #pragma omp critical
        seed = static_cast<unsigned int>(std::time(0)) + tid;
        gen.seed(seed);
        
        do {
            //  Run batch of experiments.
            sum_x = 0;
            sum_x2 = 0;
            for (int t = 0; t < tnbatch; ++t) {
                double x = dist(gen);
                double xx[2][NPTS];
                double d2 = -1;
                
                for (int i = 0; i < NPTS; i++) {
                    double x_i = dist(gen);
                    double y_i = dist(gen);
                    xx[0][i] = x_i;
                    xx[1][i] = y_i;
                    //  Assess distance between points.
                    for (int j = 0; j < i; j++) {
                        double d_x_j  = xx[0][j] - x_i;
                        double d_y_j  = xx[1][j] - y_i;
                        double d_ij2 = d_x_j*d_x_j + d_y_j*d_y_j;
                        if (d2 < 0 || d_ij2 < d2)
                            d2 = d_ij2;
                    }
                }
                x = sqrt(d2);
                sum_x  += x;
                sum_x2 += x*x;
            }
            
            //  Update global counts and test for termination.
            #pragma omp critical
            {
                f_done = (f_done || is_converged(all_sum_x, all_sum_x2, all_ntrials));
                all_sum_x   += sum_x;
                all_sum_x2  += sum_x2;
                all_ntrials += tnbatch;
                f_done = (f_done || is_converged(all_sum_x, all_sum_x2, all_ntrials));
            }
        } while (!f_done);
    }
    
    //  Compute expected value and error bars at one standard deviation.
    E_x     = all_sum_x / all_ntrials;
    E_x2    = all_sum_x2 / all_ntrials;
    sigma_x = sqrt((E_x2-E_x*E_x) / all_ntrials);
    
    //  Output the value and error bar.
    printf("With %d threads, the result was $\\bar{x}$ = %f and $\\sigma_{x}$ = %f.\n", num_threads, E_x, sigma_x);
    return 0;
}
//rewrite this using TINA?  using thread lock on master RNG? _both_

In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  monte_carlo_omp.cpp -o monte_carlo_omp

In [ ]:
!./monte_carlo_omp

---
<a id='tips'></a>
## A Few More Tips
**Environment Variables**

OpenMP permits you to specify characteristics of the system's behavior through environment variables as well, letting you tune your application for a given platform without having to deal with those idiosyncrasies in code.

`$OMP_NUM_THREADS 8/4,2`
Specifies the default number of threads to use in parallel regions.  "Having more OpenMP threads than hardware threads/cores can improve load balancing." ([forum](http://openmp.org/forum/viewtopic.php?f=3&t=1566))  (But watch out for resource conflicts and cache thrashing!)

`$OMP_DYNAMIC TRUE/FALSE`

`$OMP_NESTED TRUE/FALSE`

`$OMP_PROC_BIND TRUE/FALSE`.  With processor binding, the programmer instructs the operating system that a thread in the program should run on the same processor throughout the execution of the program.

Performance can be enhanced with processor binding, but performance degradation will occur if multiple threads are bound to the same virtual processor. ([(Oracle/Sun)](http://docs.oracle.com/cd/E24457_01/html/E21996/aewce.html#scrolltoc)) ([Hyperthreading poor performance](https://stackoverflow.com/questions/24368576/poor-performance-due-to-hyper-threading-with-openmp-how-to-bind-threads-to-core))

Binding threads to processors can result in better cache utilization, thereby reducing costly memory accesses. This is the primary motivation for binding threads to processors. ([LLNL](https://computing.llnl.gov/tutorials//openMP/index.html#Stack))

`$OMP_STACKSIZE 10M`

`$OMP_WAIT_POLICY ACTIVE/PASSIVE`

`$OMP_SCHEDULE 'STATIC'/'GUIDED,4'` ([good explanation of when to use](http://openmp.org/forum/viewtopic.php?f=3&t=83))

`$GOMP_CPU_AFFINITY "0 1 2 3 4 5 6 7"`.  Binds threads to specific CPUs.  (Differs from `$OMP_PROC_BIND` which merely binds threads to processors without switching, not specifying which ones.)  Can help performance, esp with `$OMP_PROC_BIND=TRUE`.  ([forum](https://stackoverflow.com/questions/24368576/poor-performance-due-to-hyper-threading-with-openmp-how-to-bind-threads-to-core))

**Sharing Memory-Managed STL Containers**

It's a little tricky to use memory-managed containers (STL) in OpenMP, but some clever tricks can enable it (mainly, the judicious use of `master`, `single`, and especially `critical`).  For instance, the following technique should let you create a vector in parallel and then use the `critical` keyword to populate the communal vector.

    std::vector<int> vec;
    #pragma omp parallel
    {
        std::vector<int> vec_private;
        #pragma omp for nowait //fill vec_private in parallel
        for(int i=0; i<100; i++) {
            vec_private.push_back(i);
        }
        #pragma omp critical
        vec.insert(vec.end(), vec_private.begin(), vec_private.end());
    }

**Faster `for` Loops**

The default scheduling behavior for OpenMP is to partition out the iterations between threads with a chunk size of `n_indices / n_threads`.  Thus these threads, when they reach the end of their designated workload, just idle until all threads catch up.  A better way to handle this is to invoke some automatic load balancing with `schedule(dynamic, 1)`, which forces the system to hand out new work until all of it is done at once.  ([src](http://brainvis.wustl.edu/wiki/index.php/Caret7:Development/OpenMP_tricks))

**Barriers**

The use of `barrier` should be avoided, as it can cause inefficient overhead in many cases by making all threads wait to carry out an operation.  (This goes for `ordered` as well—and don't use large `critical` regions.)

**`private` Variables**

Apparently on certain compilers, `private` has not worked properly in all cases.  Should you find this to be a problem, simply move your `private` variable declarations within the `parallel` code block, automatically rendering them private.  (This should not be a problem on contemporary compilers but may be on systems from 2011 or older.)  ([src](http://brainvis.wustl.edu/wiki/index.php/Caret7:Development/OpenMP_tricks))

    int i;
    #pragma omp parallel for private(i)
    { ... }
    
    #pragma omp parallel for
    for (int i = 0; i < N; i++) { ... }

**Loop Granularity**

Generally speaking, you will find better scalability with OpenMP if you keep loop granularity as fine as reasonable (but no finer!).  What this means is that you want to use the smallest iteration that doesn't incur sufficient thread creation overhead to make it inefficient.  This translates into two tips:  use nested OpenMP loops as appropriate; and use the `collapse` keyword to generate collapsed `for` loops.  `parallel` regions in inner loops incur overhead repeatedly and should be avoided.  ([ref](https://software.intel.com/en-us/articles/openmp-related-tips))

    #pragma omp parallel for collapse(2)
    for (i = 0; i < imax; i++)
        for (j = 0; j < jmax; j++)
            a[ j + jmax*i] = 1.; 

You want to maximize parallel regions to get good benefit from OpenMP.

**Are My Loop Iterations Independent?**

Run it backwards.  If it still works, they most likely are.

**Load Balance**

Just do it.  If practical.

**User-Defined Reduction Operations**

As of [OpenMP 4.0](http://www.openmp.org/mp-documents/OpenMP4.0.0.pdf), user-defined reduction operations are supported.  This means that instead of being limited to scalar reductions (over `int`, `double`, etc.), you can now define reduction over, say, an STL `vector<double>`, thus enabling vector mathematics to use OpenMP.  (This is gold, by the way.)

    #pragma omp declare reduction (merge : std::vector<int> : omp_out.insert(omp_out.end(), omp_in.begin(), omp_in.end())) 

(For what it may be worth, when I first wanted to test this, my installed GCC version (4.8) didn't support the feature yet.  I upgraded to GCC 4.9 [which does support OpenMP 4.0](http://www.techenablement.com/gcc-4-9-1-adds-openmp-4-0-fortran-support-for-multicore/) and then tested the code which follows as `reduce.cpp`.)

In [ ]:
%%file reduce.cpp
#include <iostream>
#include <cstdlib>
#include <omp.h>

using namespace std;

struct force_t { double x, y; };

int main(int argc, char** argv) {
    #pragma omp declare reduction (+ : force_t : omp_out.x += omp_in.x, omp_out.y += omp_in.y)
    force_t pt;
    int N = 10;
    if (argc > 1) N = atoi(argv[1]);
    
    #pragma omp parallel for reduction(+ : pt) schedule(static,1)
    for (int j = 0; j < N; j++) {
        force_t my_pt;
        my_pt.x = 1.0*j;
        my_pt.y = 2.0*j;
        pt.x += my_pt.x;
        pt.y += my_pt.y;
    }
    cout << "Using " << omp_get_max_threads() << " threads, the resultant force is (" << pt.x << "," << pt.y << ").\n";
}

In [ ]:
!g++-4.9 -Wall -fopenmp -O2 -std=c++11 -pedantic reduce.cpp -o reduce

In [ ]:
!./reduce 32

### Exercise
- Write a reduction code for the C++ STL `complex` data type addition operation.

In [ ]:
%%file reduce-complex.cpp


In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  reduce-complex.cpp -o reduce-complex

In [ ]:
!./reduce-complex 32

---
<a id='vector'></a>
## Using Vectorization (SIMD) in Practice

Vectorization (SIMD) is typically implemented by using compilation options in many modern processor/compiler workflows, however, so the scientific programmer rarely needs to directly address this operation in code.  For instance, with GCC, vectorization may be included by using the following compiler flags:

- `-O3` Full optimization typically turns on all SSE vector operations available on the target processor platform, but makes it extremely difficult to debug the code effectively and should only be used in testing or release codes ([src]()) ([ref](https://gcc.gnu.org/onlinedocs/gcc/Optimize-Options.html)).

- `-msse4.2` The current generation of x86-64 processors supports some SSE standard; in this case, the current extension set supported is SSE 4.2.  (This enables all lower SSE extensions automatically, _e.g._, `-msse3` is not necessary with `-msse4` invoked. ([src](http://stackoverflow.com/a/10687419)) ([ref](https://gcc.gnu.org/onlinedocs/gcc/Option-Summary.html)))

- `-mavx` The AVX (*A*dvanced *V*ector *E*xtensions) instruction set architecture is only supported on Intel Sandy Bridge and AMD Bulldozer processors and later models.  They extend integer commands to 256 (later 512) bits, increasing the performance of floating-point operations and vectorizable array operations. ([src](https://01.org/blogs/dfineber/2014/devops-does-processor-matter-server-side-java-applications)) 

**Caution!**  Know your architecture, however.  Specialized hardware, such as the Intel Xeon Phi, may not support a particular method.  In this case, the Xeon E3-1275v3 processor supports AVX but not SSE—it is not intended for multimedia operations, but numerical work on supercomputers.  ([src](https://communities.intel.com/thread/43942?start=0&tstart=0))

Thus, effectively using vectorization is often simply a matter of compiling your program correctly on a supporting architecture:

    gcc -O3 -msse4.2 -std=c99 -Wall -o hello_world_vector hello_world_vector.c

Consult the following references for more information.

- [(Stack Overflow:  Comparison between OpenMP and Vectorization)](https://stackoverflow.com/questions/10509872/comparison-between-openmp-and-vectorization)

- [(Stack Overflow:  Using OpenMP Stops GCC Auto Vectoising)](https://stackoverflow.com/questions/14861447/using-openmp-stops-gcc-auto-vectorising?rq=1)

- [(Performance Essentials with OpenMP 4.0 Vectorization)](https://software.intel.com/en-us/articles/performance-essentials-with-openmp-40-vectorization)

- [(SIMD Programming with GCC)](http://basis13.wordpress.com/2014/02/20/simd-programming-with-gcc-first-steps/)

---
<a id='assess'></a>
## Assessing Performance

We've gone to a fair amount of work to make our codes parallel and vectorized, but is it worth the effort?  As always, it depends.  We'll run a few sample codes below to assess their relative performance under different library and compilation conditions.

The basic function we need to do this is actually provided by `omp.h`:  `omp_get_wtime`, which has `double` resolution in seconds and so is quite adequate for timing both parallel and serial code segments.

Let's take a look at the series product problem from earlier.

In [ ]:
%%file series-time.cpp
#include <complex>
#include <iostream>
#include <cmath>
#include <omp.h>

using namespace std;

#define PI 3.1415926535

// The k-th term of the infinite product series for sine.
static complex<double> term(int k, complex<double> z) {
    return (complex<double>(1.0, 0.0) - pow(z, 2)/(pow(k, 2) * PI*PI));
}

int main(int argc, char** argv) {
    int N = 10;
    if (argc > 1) N = atoi(argv[1]);
    complex<double> z = complex<double>(PI*0.25, PI*0.25);
    if (argc > 2) z = complex<double>(atof(argv[2]), atof(argv[3]));
    
    double ser_start, ser_end, par_start, par_end;
    
    complex<double> res, i_res;
    double res_r = 1.0, res_i = 1.0;
    int j;
    
    //  Solve the problem serially first.
    ser_start = omp_get_wtime();
    for (j = 1; j < N; j++) {
        i_res = term(j, z);
        res_r *= real(i_res);
        res_i *= imag(i_res);
    }
    // note that reduction over STL data types is not yet supported
    res = complex<double>(res_r, res_i) * z;
    ser_end = omp_get_wtime();
    cout << "The serial code takes " << (ser_end - ser_start) << " s to get the answer " << res << ".\n";
    
    //  Solve the parallel problem.
    par_start = omp_get_wtime();
    #pragma omp parallel for private(i_res, j) reduction(*:res_r,res_i) schedule(static,1)
    for (j = 1; j < N; j++) {
        i_res = term(j, z);
        res_r *= real(i_res);
        res_i *= imag(i_res);
    }
    // note that reduction over STL data types is not yet supported
    res = complex<double>(res_r, res_i) * z;
    par_end = omp_get_wtime();
    cout << "The parallel code takes " << (par_end - par_start) << " s to get the answer " << res << " with " << omp_get_max_threads() << " threads.\n";
}

In [ ]:
!g++ -Wall -fopenmp -O2 -pedantic  series-time.cpp -o series-time

In [ ]:
!./series-time 1
!./series-time 10
!./series-time 100
!./series-time 1000
!./series-time 10000
!./series-time 100000
!./series-time 1000000
!./series-time 10000000
!./series-time 100000000

In this case, for small numbers of the series there is clearly a penalty to be paid for parallelizing the code (see the figure below from some data on my own machine, a 2013 MacBook Pro with quad-core 2.7 GHz Intel i7).  However, as the number of terms in the series increases, there is a crossover point and parallel computation becomes more efficient from a _wall time_ perspective.  (That is, for time as we observe it.  In processor time, the total time spent on the calculation should be multiplied by the number of processes involved.)

![](https://bytebucket.org/davis68/parallel/raw/fc24d274c088ba5087688ef25cf8e724f532319d/lessons/img/series-time.png?token=136c4e9258fff5a3714dd9fbe0f8317bb0c4dc8b)

You will find this generally true for a well-crafted parallel algorithm—small problems incur too much overhead to be worth the effort, but for large domains and arrays OpenMP can yield a handsome benefit.

---
<a id='bestprac'></a>
## Best Practices for Scientific Parallel Programming

Now this may be kind of a funny aside, but it is a concern I often had as a student.  MPI owes a lot to the local computer science and parallel programming folks on campus, so is it just a tool we prefer because it is "in-house"?  There are tools that get talked about a lot, as well as advanced research projects that offer interesting features but are not widely adopted, like [CHARM++](http://charm.cs.uiuc.edu/) or [UPC](http://upc.lbl.gov/).  No, MPI is used everywhere and really is the standard for supercomputing today.

### The Process of Creating a Parallel Program

**Matrix Relaxation**

The best parallel algorithm is oten not the best sequential algorithm.  For instance, in matrix solution, we can select from Jacobi relaxation or Gauss–Seidel relaxation.  [Jacobi relaxation](https://en.wikipedia.org/wiki/Jacobi_method) is a naïve algorithm to solve a matrix equation iteratively by inverting only the diagonal elements and modifying an initial guess progressively until a solution within certain tolerances is found.  [Gauss–Seidel relaxation](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method) is formally a better algorithm, since it also takes advantage of updated data points from _the current iteration_ to improve the subsequent data points in-flight as well.  However, Jacobi relaxation, relying exclusively on a prior iteration, can be sensibly implemented in parallel while Gauss–Seidel iteration can not.

**Prefix Summation**

Consider also summing vector elements of $A$ such that each resulting element $B_i$ is the sum over all elements prior to index $i$:
$$\begin{eqnarray} B_{0} = & A_{0} \\ B_i = & \sum_{j = 1}^i A_j \text{.} \end{eqnarray}$$

In serial, this algorithm requires $N = P$ stages.  In parallel, this can be accomplished with a _scan_ operation in $\log P$ stages.  ([ref1](https://en.wikipedia.org/wiki/Prefix_sum)) ([ref2](http://http.developer.nvidia.com/GPUGems3/gpugems3_ch39.html))

<table>
<tr>
<td><h3>Serial Version</h3> <img src="./img/prefix-serial.png"/></td>
<td><h3>Parallel Version</h3> <img src="./img/prefix-parallel.png"/></td>
</tr>
</table>

**Basic Process**

- Determine algorithms
- Decide on decomposition of computation into work and data units
- Decide on assignment of these units to processors
- Express decisions in language available for the machine

**Tips**

- Architect a new parallel top-level structure rather than parallelizing a sequential program
- Insert existing sequential code where appropriate
- Refactor a portion of the sequential code into a parallel structure
- Rewrite the remainder
- Express algorithms in terms of $N$ and $P$.
- Visualize data flow, not control flow, to reveal dependencies.

---
<a id='conc'></a>
## Conclusion

There's a lot more to OpenMP than what we could cover in this brief introduction.  Here are a few resources to learn more about OpenMP and its applications.

- Open MP 4.0 ([standard](http://www.openmp.org/mp-documents/OpenMP4.0.0.pdf)) ([examples](http://openmp.org/mp-documents/OpenMP_Examples_4.0.1.pdf))
- Lawrence Livermore National Laboratory [tutorial](https://computing.llnl.gov/tutorials/openMP/)
- Archer Advanced OpenMP ([workshop](http://www.archer.ac.uk/training/course-material/2014/05/AdvancedOpenMP_Oxford/))
- [32 Traps for OpenMP Developers](http://www.codeguru.com/cpp/cpp/cpp_mfc/general/article.php/c15419/32-OpenMP-Traps-for-C-Developers.htm) (dated now but still good basic troubleshooting)

---
<a id='refs'></a>
## References
- [NCSA CI-Tutor](http://www.citutor.org/login.php).
- [LLNL tutorial](https://computing.llnl.gov/tutorials/openMP/)
- [UIUC CS Introduction to OpenMP](http://www.cs.uiuc.edu/class/sp06/cs232/section/disc7.pdf)
- [OpenMP with MPI](http://www.slideserve.com/MikeCarlo/parallel-programming-in-c-with-mpi-and-openmp-)
- [Using OpenMP](https://mitpress.mit.edu/books/using-openmp) (book)
- [Guide Into Using OpenMP](http://bisqwit.iki.fi/story/howto/openmp/)
- [Thoughts on OpenMP in Scientific Computing](http://www.dalkescientific.com/writings/diary/archive/2012/01/16/my_views_on_openmp.html)
- [Official OpenMP Code Examples](http://openmp.org/mp-documents/OpenMP_Examples_4.0.1.pdf)

---
<a id='credits'></a>
## Credits

Neal Davis developed these materials for [Computational Science and Engineering](http://cse.illinois.edu/) at the University of Illinois at Urbana–Champaign.

<img src="http://i.creativecommons.org/l/by/4.0/88x31.png" align="left">
This content is available under a [Creative Commons Attribution 4.0 Unported License](https://creativecommons.org/licenses/by/4.0/).

[![](https://bytebucket.org/davis68/resources/raw/f7c98d2b95e961fae257707e22a58fa1a2c36bec/logos/baseline_cse_wdmk.png?token=be4cc41d4b2afe594f5b1570a3c5aad96a65f0d6)](http://cse.illinois.edu/)

---
## Code Snippets for Teaching

In [ ]:
import numpy as np
from numpy import pi, cos, linspace
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rcParams['figure.figsize']=[18,3]
x = linspace(pi-0.05, 2.5*pi+0.02, 1001)
f = cos(x)/(2+cos(x))
f[0] = f[-1] = 0
fig = plt.figure()
ax = fig.add_subplot(111)
ax.fill(x, f, color='cornflowerblue')
ax.set_xlim(pi, 2.5*pi)
ax.set_ylim(-1.0, 1.0)
unit   = 0.5
x_tick = np.arange(1, 2.5+unit, unit)
x_label= [r"$\pi$", r"$\frac{3\pi}{2}$", r"$2\pi$", r"$\frac{5\pi}{2}$"]
ax.set_xticks(x_tick*np.pi)
ax.set_xticklabels(x_label, fontsize=16)
plt.show()

In [ ]:
x = linspace(0, 2.02, 1001)
f = np.sinh(x) / np.cosh(x) - 1.0
f[0] = f[-1] = 0
fig = plt.figure()
ax = fig.add_subplot(111)
ax.fill(x, f, color='maroon')
ax.set_xlim(0.0, 2.0)
ax.set_ylim(-1.2,0.2)
plt.show()

In [ ]:
%%file reduce-complex.cpp
#include <complex>
#include <iostream>
#include <cstdlib>
#include <omp.h>

int main(int argc, char** argv) {
    #pragma omp declare reduction (+ : std::complex<double> : omp_out += omp_in)
    int N = 10;
    if (argc > 1) N = atoi(argv[1]);
    std::complex<double> val;
    
    #pragma omp parallel for reduction(+ : val) schedule(dynamic,1)
    for (int j = 0; j < N; j++) {
        std::complex<double> my_val = std::complex<double>(2.0,2.0);
        val += my_val;
    }
    std::cout << "Using " << omp_get_max_threads() << " threads, the summed complex value is " << val.real() << "+" << val.imag() << "i.\n";
}